In [1]:
import os
import numpy as np
from scipy import ndimage as ndi
from scipy.stats import poisson
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd

from analyfits import misc
from analyfits import manipufits
from analyfits import datafits

In [8]:
# Cargo una lista con los directorios de todas las imágenes.
DIR = "/home/igna/Escritorio/Tesis2021/Archivos/T123K"
#DIR_win = "D:\ignag\Igna\Facultad\Tesis2021\Archivos\T123K_B"
os.chdir(DIR)
fits_imgs_list = misc.fits_finder(DIR)

In [9]:
len(fits_imgs_list)

925

# 1) $\mu_{T}$
## Calculo la contribución total en el primer borde cercano a clusters

In [4]:
ones = 0
zeros = 0
for i, img in enumerate(fits_imgs_list):
    # Original
    img_original = manipufits.ADU2e(img)[0][:, 8:451]
    # 2 o más eventos
    img_clusters = manipufits.single_fits2double_fits(img)[1][:, 8:451]
    # Estructura
    struct = [[1,1,1],
              [1,1,1],
              [1,1,1]]
    
    # Máscara (dilato 1 vez)
    mask = ndi.morphology.binary_dilation(img_clusters, structure=struct)
    # Resta: A la máscara le saco los pixeles donde hay eventos de 2 o más
    mask = np.where(img_clusters!=0, 0, mask)
    # A la imagen original le saco todos los eventos menos los que cayeron en la mask
    img_1e = np.where(mask!=0, img_original, 0)
    # acumulo
    ones += img_1e[img_1e!=0].size
    zeros += np.sum(mask)
    porcentaje = (i+1)*100/len(fits_imgs_list)
    print(f"\rCompletado {porcentaje:.2f}%",end= "")
mu_T, error_mu_T = ones/zeros, np.sqrt((1/np.sqrt(ones))**2 + (1/np.sqrt(zeros))**2)*(ones/zeros)

Completado 100.00%

# 2) $\mu_{bkg}$
## Calculo la contribución del background a la esperanza total de eventos de 1 electron usando unicamente los eventos en el segundo borde cercano a los clusters

In [5]:
ones = 0
zeros = 0
for i, img in enumerate(fits_imgs_list):
    # Original
    img_original = manipufits.ADU2e(img)[0][:, 8:451]
    # 2 o más eventos
    img_clusters = manipufits.single_fits2double_fits(img)[1][:, 8:451]
    # Estructura
    struct = [[1,1,1],
              [1,1,1],
              [1,1,1]]
    
    # Máscara1 (dilato 1 vez)
    mask_1 = ndi.morphology.binary_dilation(img_clusters, structure=struct)
    # máscara2 (dilato 1 vez mask_1)
    mask_2 = ndi.morphology.binary_dilation(mask_1, structure=struct)
    # Resta: A la máscara2 le saco los pixeles de mask_1
    mask_2 = np.where(mask_1!=0, 0, mask_2)
    # A la imagen original le saco todos los eventos menos los que cayeron en la mask
    img_1e = np.where(mask_2!=0, img_original, 0)
    # acumulo
    ones += img_1e[img_1e!=0].size
    zeros += np.sum(mask_2)
    porcentaje = (i+1)*100/len(fits_imgs_list)
    print(f"\rCompletado {porcentaje:.2f}%",end= "")
mu_bkg, error_mu_bkg = ones/zeros, np.sqrt((1/np.sqrt(ones))**2 + (1/np.sqrt(zeros))**2)*(ones/zeros)

Completado 100.00%

# 3) $\mu_{g}$
## Calculo la esperanza para los eventos de un electrón genuinos

In [6]:
mu_g = mu_T - mu_bkg
error_mu_g = error_mu_T - error_mu_bkg

In [7]:
print(f"mu_T = {mu_T:.10f} +/- {error_mu_T:.10f}")
print(f"mu_bkg = {mu_bkg:.10f} +/- {error_mu_bkg:.10f}")
print(f"mu_g = {mu_g:.10f} +/- {error_mu_g:.10f}")

mu_T = 0.2040032554 +/- 0.0002427943
mu_bkg = 0.1899700260 +/- 0.0002051048
mu_g = 0.0140332294 +/- 0.0000376895
